In [7]:
import pathlib
import numpy as np
import tensorflow as tf
import sys
import pandas as pd
sys.path.append("..")
from utils.preprocess import *

In [8]:
# class ExportModel(tf.Module):
#   def __init__(self, model):
#     self.model = model

#     # Accept either a string-filename or a batch of waveforms.
#     # YOu could add additional signatures for a single wave, or a ragged-batch. 
#     self.__call__.get_concrete_function(
#         x=tf.TensorSpec(shape=(), dtype=tf.string))
#     self.__call__.get_concrete_function(
#        x=tf.TensorSpec(shape=[None, 16000], dtype=tf.float32))


#   @tf.function
#   def __call__(self, x):
#     # If they pass a string, load the file and decode it. 
#     if x.dtype == tf.string:
#       x = tf.io.read_file(x)
#       x, _ = tf.audio.decode_wav(x, desired_channels=1, desired_samples=16000,)
#       x = tf.squeeze(x, axis=-1)
#       x = x[tf.newaxis, :]

#     x = get_spectrogram(x)  
#     result = self.model(x, training=False)

#     class_ids = tf.argmax(result, axis=-1)
#     class_names = tf.gather(label_names, class_ids)
#     return {'predictions':result,
#             'class_ids': class_ids,
#             'class_names': class_names}

In [9]:
# def get_spectrogram(waveform):
#   # Convert the waveform to a spectrogram via a STFT.
#   spectrogram = tf.signal.stft(
#       waveform, frame_length=255, frame_step=128)
#   # Obtain the magnitude of the STFT.
#   spectrogram = tf.abs(spectrogram)
#   # Add a `channels` dimension, so that the spectrogram can be used
#   # as image-like input data with convolution layers (which expect
#   # shape (`batch_size`, `height`, `width`, `channels`).
#   spectrogram = spectrogram[..., tf.newaxis]
#   return spectrogram

# def preprocess(x):
#     x = tf.io.read_file(x)
#     x, _ = tf.audio.decode_wav(x, desired_channels=1, desired_samples=16000,)
#     x = tf.squeeze(x, axis=-1)
#     x = x[tf.newaxis, :]

#     x = get_spectrogram(x)  
#     return x

In [10]:
# export = ExportModel(model)
# export(tf.constant(str(data_dir/'no/01bb6a2a_nohash_0.wav')))

In [11]:
# DATASET_PATH = 'data/mini_speech_commands'

# data_dir = pathlib.Path(DATASET_PATH)

In [12]:
import os

In [13]:
input_directory = "../assets/pdiot_submission/s2029523"

data_list = []
label_list = []
total_rows = 0

for file in os.listdir(input_directory):
    if file.endswith(".csv") and "unprocessed" not in file:
        filename = os.path.join(input_directory, file)
        label, _, _ = extract_activity_and_status(filename)

        df = pd.read_csv(filename, usecols=[1,2,3])

        # Determine the number of rows to take from this file
        rows_to_take = min(700 - total_rows, len(df))
        
        # Update the df to only contain the necessary rows and update our counter
        df = df.head(rows_to_take)
        label_list.append(label)
        data_list.append(df)

# all_data = pd.concat(data_list, ignore_index=True)
print(data_list[0])

      accel_x   accel_y   accel_z
0    0.070068 -1.013977  0.104187
1   -0.044434 -0.960999  0.007507
2    0.000244 -0.932434 -0.016663
3   -0.043457 -0.956360  0.100525
4    0.015381 -1.002502  0.049988
..        ...       ...       ...
695 -0.003662 -0.681946 -0.146545
696 -0.065430 -0.805481 -0.159485
697 -0.011719 -0.970520 -0.095520
698 -0.107178 -1.269348 -0.158752
699 -0.046387 -1.302307 -0.083557

[700 rows x 3 columns]


In [14]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Normalizing sensor readings
scaler = StandardScaler()

sensor_columns = ['accel_x', 'accel_y', 'accel_z']

scaler.fit(data_list[0][sensor_columns])

for i in range(len(data_list)):
    data_list[i][sensor_columns] = scaler.transform(data_list[i][sensor_columns])


In [15]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert data and labels to numpy arrays
X = np.array([df.values for df in data_list])
X = np.stack(X)
y = LabelEncoder().fit_transform(label_list)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert X_train to float32
X_train = X_train.astype('float32')

In [16]:
import tensorflow as tf
import numpy as np

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="../exported_models/model_combined.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the audio file
# file_path = str(data_dir/'no/01bb6a2a_nohash_0.wav')
# data = preprocess(file_path)

# Ensure the input data is the correct shape and type
input_shape = input_details[0]['shape']
input_data = np.array(X_train[0], dtype=input_details[0]['dtype']).reshape(input_shape)

# Set the tensor and invoke the interpreter
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)


[[6.2492833e-04 1.3478450e-03 7.0554376e-01 1.0452529e-02 3.5982503e-04
  4.4327203e-02 4.6993850e-04 6.6712382e-03 7.1605816e-02 6.5388018e-03
  1.4824677e-01 3.8113627e-03]]
